In [277]:
# download from gefs-aws into split gribs using curl
# lot of hard coding needs to be changed to customize
# see cell with # HARDCODED REGEX


import boto3
from botocore import UNSIGNED
from botocore.config import Config

# CONFIG
bucket_name = "noaa-gefs-pds"
prefix = 'gefs.202406'
max_hour = 384

model_url_folder = 'https://noaa-gefs-pds.s3.amazonaws.com/'
model_data_folder = '/home/db/Documents/JRPdata/hot-july/gefs'
# the identifiable string in the .idx files to extract split gribs (should be in format ':shortname:desc:')
params = [':TMP:2 m above ground:']
# how long to pause between downloads as to not hammer server
sleep_time = 1

s3_client = boto3.client('s3', config=Config(signature_version=UNSIGNED))

def get_s3_keys(bucket, s3_client, prefix = '', foldersOnly=True):
    """
    Generate the keys in an S3 bucket.
n fo
    :param bucket: Name of the S3 bucket.
    :param prefix: Only fetch keys that start with this prefix (optional).
    """
    kwargs = {'Bucket': bucket}
    if isinstance(prefix, str):
        kwargs['Prefix'] = prefix
    if foldersOnly:
        kwargs['Delimiter'] ='/'
    while True:
        resp = s3_client.list_objects_v2(**kwargs)
        print(prefix)
        print(resp)
        if foldersOnly:
            for obj in resp['CommonPrefixes']:
                    key = obj['Prefix']
                    if key.startswith(prefix):
                        yield key
        else:
            for obj in resp['Contents']:
                key = obj['Key']
                if key.startswith(prefix):
                    yield key
        try:
            kwargs['ContinuationToken'] = resp['NextContinuationToken']
        except KeyError:
            break

def get_folders(folder_path):
    keys = get_s3_keys(bucket_name,s3_client,prefix=folder_path)
    return list(keys)

def get_files(folder_path):
    keys = get_s3_keys(bucket_name,s3_client,prefix=folder_path, foldersOnly=False)
    return list(keys)

In [72]:
folders = get_folders('gefs.202406')

for init_date in folders:
    init_times = get_folders(init_date)
    print(prods)
    for init_time in init_times:
        prod = get_folders(init_time)
        print(prod)
        for prod in prods:
            if 'atmos' in prod:
        break
    break

['gefs.20240601/00/', 'gefs.20240601/06/', 'gefs.20240601/12/', 'gefs.20240601/18/']
['gefs.20240601/00/atmos/', 'gefs.20240601/00/chem/', 'gefs.20240601/00/wave/']


In [67]:
print(folders)

['gefs.20240601/', 'gefs.20240602/', 'gefs.20240603/', 'gefs.20240604/', 'gefs.20240605/', 'gefs.20240606/', 'gefs.20240607/', 'gefs.20240608/', 'gefs.20240609/', 'gefs.20240610/', 'gefs.20240611/', 'gefs.20240612/', 'gefs.20240613/']


In [54]:
folders = get_folders('gefs.202406')

In [74]:
prod0 = prod

In [76]:
prods = prod0
for prod in prods:
    if 'atmos' in prod:
        print(prod)

gefs.20240601/00/atmos/


In [85]:
folders = get_folders('gefs.202406**')

gefs.202406**
{'ResponseMetadata': {'RequestId': '7YRG6XN6VZE9M0VA', 'HostId': '+4xEUPLzxCVw+sPX25LGYSHpFVzoO+9REw2x3RjITJVDbMlkxHt8DFsT50zpHQqUo8CvF44q8hU=', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amz-id-2': '+4xEUPLzxCVw+sPX25LGYSHpFVzoO+9REw2x3RjITJVDbMlkxHt8DFsT50zpHQqUo8CvF44q8hU=', 'x-amz-request-id': '7YRG6XN6VZE9M0VA', 'date': 'Thu, 13 Jun 2024 17:35:18 GMT', 'x-amz-bucket-region': 'us-east-1', 'content-type': 'application/xml', 'transfer-encoding': 'chunked', 'server': 'AmazonS3'}, 'RetryAttempts': 0}, 'IsTruncated': False, 'Name': 'noaa-gefs-pds', 'Prefix': 'gefs.202406**', 'Delimiter': '/', 'MaxKeys': 1000, 'EncodingType': 'url', 'KeyCount': 0}


KeyError: 'CommonPrefixes'

In [231]:
import re

# this code block from nickorka@github: https://github.com/aws/aws-cli/issues/3784#issuecomment-530432844
# regex starts after prefix
def search_s3_regex(results, bucket, s3_client, prefix, regex_path):
    wc_parts = regex_path.split('/')
    if len(wc_parts) == 1 and len(wc_parts[0]) == 0:
        results.append(prefix)
        return
    else:
        regex = re.compile(wc_parts[0])
        next_regex_path = '/'.join(wc_parts[1:])
        paginator = s3_client.get_paginator('list_objects')
        
        result = paginator.paginate(Bucket=bucket, Delimiter='/', Prefix=prefix)
        for pref in result.search('CommonPrefixes'):
            if pref is None:
                # check files
                for k in result.search('Contents'):
                    res = k.get('Key')
                    search_prefix = res if len(prefix) == 0 else res.split(prefix)[1]
                    if re.match(regex, search_prefix):
                        results.append(res)
            else:
                # check paths
                res = pref.get('Prefix')
                search_prefix = res if len(prefix) == 0 else res.split(prefix)[1]
                if re.match(regex, search_prefix):
                    search_s3_regex(results, bucket, s3_client, res, next_regex_path)
                    

In [232]:

# HARDCODED REGEX



res =[]
# regex starts after prefix
# get gefs bc 0.5 deg in june

search_s3_regex(res, bucket_name, s3_client, 'gefs.202406', '[0-9]{2}/[0-9]{2}/atmos/pgrb2ap5/geavg.t[0-9]{2}z.pgrb2a.0p50.f[0-9].*')


In [233]:

idxlist = []
griblist = []

for f in res:
    r = re.search(r'.*/geavg.t[0-9]{2}z.pgrb2a.0p50.f(?P<time_step>[0-9][\d]{2})(?P<ext>\..*)?', f)

    if r:
        num_groups = len(r.groups())
        if num_groups > 0:
            isIDX = False
            if num_groups == 2 and r['ext'] == '.idx':
                isIDX = True

            if int(r['time_step']) <= max_hour:
                # valid idx or grib file
                #https://noaa-gefs-pds.s3.amazonaws.com/gefs.20240613/00/atmos/pgrb2ap5/geavg.t00z.pgrb2a.0p50.f000
                if isIDX:
                    idxlist.append(f)
                else:
                    griblist.append(f)

In [257]:
idx_files_by_model_times = dict()
grib_files_by_model_times = dict()
for idx in idxlist:
    r = re.search(r'gefs.(?P<model_date>[0-9]{8})/(?P<model_hour>[0-9]{2})/atmos/pgrb2ap5/geavg.t[0-9]{2}z.pgrb2a.0p50.f(?P<time_step>[0-9][\d]{2})(?P<ext>\..*)?', idx)
    num_groups = len(r.groups())
    if num_groups == 4:
        model_time = f"{r['model_date']}_{r['model_hour']}"
        if model_time not in idx_files_by_model_times:
            idx_files_by_model_times[model_time] = []
        idx_files_by_model_times[model_time].append(idx)
    #url_folder = f'{model_url_folder}/gefs.{model_date}/{model_hour}/atmos/pgrb2ap5/'
    
for grib in griblist:
    r = re.search(r'gefs.(?P<model_date>[0-9]{8})/(?P<model_hour>[0-9]{2})/atmos/pgrb2ap5/geavg.t[0-9]{2}z.pgrb2a.0p50.f(?P<time_step>[0-9][\d]{2})(?P<ext>\..*)?', grib)
    num_groups = len(r.groups())
    if num_groups == 4:
        model_time = f"{r['model_date']}_{r['model_hour']}"
        if model_time not in grib_files_by_model_times:
            grib_files_by_model_times[model_time] = []
        grib_files_by_model_times[model_time].append(grib)

In [261]:
num_idx_files_by_model_times = dict()
num_grib_files_by_model_times = dict()
for model_time, idxfiles in idx_files_by_model_times.items():
    num_idx_files_by_model_times[model_time] = len(idxfiles)
for model_time, gribfiles in grib_files_by_model_times.items():
    num_grib_files_by_model_times[model_time] = len(gribfiles)

In [ ]:
num_idx_files_by_model_times = sorted(num_idx_files_by_model_times)
num_grib_files_by_model_times = sorted(num_grib_files_by_model_times)
if num_idx_files_by_model_times == num_grib_files_by_model_times:
    print("There are an equal number of idx/grib files")
# sanity check to make sure there are no incomplete directories

In [266]:
# cribbed from download.py / auto_download.py
import os
import subprocess
import time
import re
import requests

from datetime import datetime, timedelta

print_level_importance = 3

def print_level(level, warning_str):
    if level <= print_level_importance:
        print(warning_str)

def find_matching_lines_gfs(idx_file_path, params):
    matching_lines = []
    all_lines = []
    with open(idx_file_path, 'r') as file:
        lines = file.readlines()
        for i, line in enumerate(lines):
            if any(param in line for param in params):
                matching_lines.append((i, line))
            if ':' in line:
                all_lines.append((i, line))
    return all_lines, matching_lines

def generate_curl_commands_gfs(timestamp_prefix, idx_file_name, url_folder, url_base_file_name, output_dir, params):
    output_dir_timestamp = os.path.join(output_dir, f'gefs.{timestamp_prefix}')
    idx_file_path = os.path.join(output_dir, f'gefs.{timestamp_prefix}', f'{timestamp_prefix}_{idx_file_name}')
    #os.makedirs(output_dir_timestamp, exist_ok=True)
    if not os.path.exists(idx_file_path):
        url_idx = f'{url_folder}{idx_file_name}'
        curl_command = f"curl --create-dirs -y 30 -Y 30 -f -v -s {url_idx} -o {idx_file_path}"
        print_level(4, curl_command)

        # Run the subprocess and capture its output
        result = subprocess.run(curl_command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        time.sleep(sleep_time)
        if result.returncode != 0:
            print_level(3, f"Warning: Command failed with exit code {result.returncode}")
            print_level(3, f"Command: {curl_command}")
            print_level(3, f"Error output: {result.stderr.decode('utf-8')}")
            try:
                # clear directory if no files ready yet
                os.rmdir(output_dir_timestamp)
            except:
                pass

            return -1

    all_lines, matching_lines = find_matching_lines_gfs(idx_file_path, params)
    total_lines = len(all_lines)

    for i, (line_num, line) in enumerate(matching_lines):
        fields = line.split(':')
        start = int(fields[1])
        if i == total_lines - 1:
            range_str = f"{start}-"
            # TODO, right now always overwrite last file? no, change from getting the http size of the url first and setting that as end-1
            size = -1
        else:
            next_line_fields = all_lines[line_num + 1][1].split(':')
            end = int(next_line_fields[1]) - 1
            range_str = f"{start}-{end}"
            size = end - start + 1

        param_name = fields[3].replace(':', '_').replace(' ', '_').lower()
        param_level = fields[4].replace(':', '_').replace(' ', '_').lower()
        output_file = os.path.join(output_dir_timestamp, f"{timestamp_prefix}_{os.path.basename(url_base_file_name)}_{param_name}_{param_level}.grib2")

        overwrite_or_download = True

        if os.path.exists(output_file):
            if os.path.getsize(output_file) == size:
                overwrite_or_download = False

        if overwrite_or_download:
            url = f'{url_folder}{url_base_file_name}'
            curl_command = f"curl --create-dirs -y 30 -Y 30 -f -v -s -r {range_str} {url} -o {output_file}"
            print_level(4, curl_command)

            # Run the subprocess and capture its output
            result = subprocess.run(curl_command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
            time.sleep(sleep_time)
            if result.returncode != 0:
                print_level(3, f"Warning: Command failed with exit code {result.returncode}")
                print_level(3, f"Command: {curl_command}")
                print_level(3, f"Error output: {result.stderr.decode('utf-8')}")
                return -2

    return 0


In [279]:
for model_time, gribfiles in grib_files_by_model_times.items():
    timestamp_prefix = model_time
    for gribfile in gribfiles:
        pathsplit = gribfile.split('/')
        url_folder_subdir = "/".join(pathsplit[0:-1])
        url_folder = f'{model_url_folder}{url_folder_subdir}/'
        url_base_file_name = "/".join(pathsplit[-1:])
        idx_file_name = f"{url_base_file_name}.idx"
        output_dir = model_data_folder
        r = generate_curl_commands_gfs(timestamp_prefix, idx_file_name, url_folder, url_base_file_name, output_dir, params)

In [ ]:
# check in bash shell the smallest grib file:
# ls -alS * | grep grib2 | grep 0 | sort -r | tail
